# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor

Using TensorFlow backend.
/Users/timsennett/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/timsennett/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/timsennett/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [2]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data.dropna(axis=0, subset=['total_pymnt'], inplace=True)

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [3]:
# Print the dimensions of data 
data.shape

(42535, 16)

## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [4]:
# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [5]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer()

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_imputed), columns=cont_features)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_imputed), columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [6]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.fillna('missing', inplace=True)
X_test_cat.fillna('missing', inplace=True)


# OneHotEncode categorical variables
ohe = OneHotEncoder()

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

Run the below cell to combine the numeric and categorical features. 

In [7]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [8]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [9]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [10]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model,
                                 epochs=50,
                                 batch_size=256)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [11]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1,
                                      X_train_all,
                                      y_train_scaled,
                                      cv=5)

Epoch 1/50
23819/23819 [==============================] - 0s 17us/step - loss: 0.5846 - mean_squared_error: 0.5846
Epoch 2/50
23819/23819 [==============================] - 0s 8us/step - loss: 0.2697 - mean_squared_error: 0.2697
Epoch 3/50
23819/23819 [==============================] - 0s 10us/step - loss: 0.2476 - mean_squared_error: 0.2476
Epoch 4/50
23819/23819 [==============================] - 0s 8us/step - loss: 0.2360 - mean_squared_error: 0.2360
Epoch 5/50
23819/23819 [==============================] - 0s 7us/step - loss: 0.2286 - mean_squared_error: 0.2286
Epoch 6/50
23819/23819 [==============================] - 0s 7us/step - loss: 0.2238 - mean_squared_error: 0.2238
Epoch 7/50
23819/23819 [==============================] - 0s 8us/step - loss: 0.2200 - mean_squared_error: 0.2200
Epoch 8/50
23819/23819 [==============================] - 0s 9us/step - loss: 0.2173 - mean_squared_error: 0.2173
Epoch 9/50
23819/23819 [==============================] - 0s 8us/step - loss: 0.2151 -

23819/23819 [==============================] - 0s 7us/step - loss: 0.2135 - mean_squared_error: 0.2135
Epoch 23/50
23819/23819 [==============================] - 0s 6us/step - loss: 0.2132 - mean_squared_error: 0.2132
Epoch 24/50
23819/23819 [==============================] - 0s 9us/step - loss: 0.2127 - mean_squared_error: 0.2127
Epoch 25/50
23819/23819 [==============================] - 0s 9us/step - loss: 0.2124 - mean_squared_error: 0.2124
Epoch 26/50
23819/23819 [==============================] - 0s 10us/step - loss: 0.2120 - mean_squared_error: 0.2120
Epoch 27/50
23819/23819 [==============================] - 0s 8us/step - loss: 0.2118 - mean_squared_error: 0.2118
Epoch 28/50
23819/23819 [==============================] - 0s 7us/step - loss: 0.2114 - mean_squared_error: 0.2114
Epoch 29/50
23819/23819 [==============================] - 0s 6us/step - loss: 0.2111 - mean_squared_error: 0.2111
Epoch 30/50
23819/23819 [==============================] - 0s 17us/step - loss: 0.2107 - me

23819/23819 [==============================] - 0s 8us/step - loss: 0.1981 - mean_squared_error: 0.1981
Epoch 44/50
23819/23819 [==============================] - ETA: 0s - loss: 0.1998 - mean_squared_error: 0.19 - 0s 8us/step - loss: 0.1979 - mean_squared_error: 0.1979
Epoch 45/50
23819/23819 [==============================] - 0s 9us/step - loss: 0.1976 - mean_squared_error: 0.1976
Epoch 46/50
23819/23819 [==============================] - 0s 7us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 47/50
23819/23819 [==============================] - 0s 7us/step - loss: 0.1973 - mean_squared_error: 0.1973
Epoch 48/50
23819/23819 [==============================] - 0s 7us/step - loss: 0.1970 - mean_squared_error: 0.1970
Epoch 49/50
23819/23819 [==============================] - 0s 8us/step - loss: 0.1970 - mean_squared_error: 0.1970
Epoch 50/50
23819/23819 [==============================] - 0s 8us/step - loss: 0.1973 - mean_squared_error: 0.1973
Epoch 1/50
23819/23819 [================

23820/23820 [==============================] - 0s 8us/step - loss: 0.2129 - mean_squared_error: 0.2129
Epoch 14/50
23820/23820 [==============================] - 0s 8us/step - loss: 0.2119 - mean_squared_error: 0.2119
Epoch 15/50
23820/23820 [==============================] - 0s 7us/step - loss: 0.2109 - mean_squared_error: 0.2109
Epoch 16/50
23820/23820 [==============================] - 0s 8us/step - loss: 0.2101 - mean_squared_error: 0.2101
Epoch 17/50
23820/23820 [==============================] - 0s 8us/step - loss: 0.2093 - mean_squared_error: 0.2093
Epoch 18/50
23820/23820 [==============================] - 0s 7us/step - loss: 0.2086 - mean_squared_error: 0.2086
Epoch 19/50
23820/23820 [==============================] - 0s 8us/step - loss: 0.2081 - mean_squared_error: 0.2081
Epoch 20/50
23820/23820 [==============================] - 0s 8us/step - loss: 0.2075 - mean_squared_error: 0.2075
Epoch 21/50
23820/23820 [==============================] - 0s 8us/step - loss: 0.2070 - mean

- Find the RMSE on train data 

In [12]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_baseline_preds))

0.45280311990784616

- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [13]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
np.sqrt(mean_squared_error(y_train, baseline_preds))

4124.37511208913

## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [14]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(10, activation='relu'))
    
    # Third hidden layer
    model.add(layers.Dense(10, activation='relu'))
    
    # Fourth hidden layer
    model.add(layers.Dense(10, activation='relu'))
    
    # Fifth hidden layer
    model.add(layers.Dense(10, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

In [15]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model,
                                 epochs=150,
                                 batch_size=256,
                                 )

In [16]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2,
                                          X_train_all,
                                          y_train_scaled,
                                          cv=5)

Epoch 1/150
23819/23819 [==============================] - 0s 19us/step - loss: 0.7925 - mean_squared_error: 0.7925
Epoch 2/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.3440 - mean_squared_error: 0.3440
Epoch 3/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.2841 - mean_squared_error: 0.2841
Epoch 4/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.2598 - mean_squared_error: 0.2598
Epoch 5/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2454 - mean_squared_error: 0.2454
Epoch 6/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2351 - mean_squared_error: 0.2351
Epoch 7/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.2286 - mean_squared_error: 0.2286
Epoch 8/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2242 - mean_squared_error: 0.2242
Epoch 9/150
23819/23819 [==============================] - 0s 8us/step - loss

23819/23819 [==============================] - 0s 8us/step - loss: 0.1993 - mean_squared_error: 0.1993
Epoch 71/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1992 - mean_squared_error: 0.1992
Epoch 72/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1997 - mean_squared_error: 0.1997
Epoch 73/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1991 - mean_squared_error: 0.1991
Epoch 74/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.1989 - mean_squared_error: 0.1989
Epoch 75/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1990 - mean_squared_error: 0.1990
Epoch 76/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1989 - mean_squared_error: 0.1989
Epoch 77/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1988 - mean_squared_error: 0.1988
Epoch 78/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.198

23819/23819 [==============================] - 0s 8us/step - loss: 0.1964 - mean_squared_error: 0.1964
Epoch 141/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1962 - mean_squared_error: 0.1962
Epoch 142/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1963 - mean_squared_error: 0.1963
Epoch 143/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1961 - mean_squared_error: 0.1961
Epoch 144/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.1964 - mean_squared_error: 0.1964
Epoch 145/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.1960 - mean_squared_error: 0.1960
Epoch 146/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1976 - mean_squared_error: 0.1976
Epoch 147/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1960 - mean_squared_error: 0.1960
Epoch 148/150
23819/23819 [==============================] - 0s 8us/step - 

23819/23819 [==============================] - 0s 8us/step - loss: 0.2011 - mean_squared_error: 0.2011
Epoch 61/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2011 - mean_squared_error: 0.2011
Epoch 62/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2012 - mean_squared_error: 0.2012
Epoch 63/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.2009 - mean_squared_error: 0.2009
Epoch 64/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.2010 - mean_squared_error: 0.2010
Epoch 65/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2008 - mean_squared_error: 0.2008
Epoch 66/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2007 - mean_squared_error: 0.2007
Epoch 67/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.2007 - mean_squared_error: 0.2007
Epoch 68/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.200

23819/23819 [==============================] - 0s 13us/step - loss: 0.1977 - mean_squared_error: 0.1977
Epoch 131/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 132/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1976 - mean_squared_error: 0.1976
Epoch 133/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1976 - mean_squared_error: 0.1976
Epoch 134/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1974 - mean_squared_error: 0.1974
Epoch 135/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1975 - mean_squared_error: 0.1975
Epoch 136/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1974 - mean_squared_error: 0.1974
Epoch 137/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1974 - mean_squared_error: 0.1974
Epoch 138/150
23819/23819 [==============================] - 0s 10us/step -

...

23819/23819 [==============================] - 0s 8us/step - loss: 0.1951 - mean_squared_error: 0.1951
Epoch 121/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1952 - mean_squared_error: 0.1952
Epoch 122/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1948 - mean_squared_error: 0.1948
Epoch 123/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1951 - mean_squared_error: 0.1951
Epoch 124/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1954 - mean_squared_error: 0.1954
Epoch 125/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1953 - mean_squared_error: 0.1953
Epoch 126/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1948 - mean_squared_error: 0.1948
Epoch 127/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1945 - mean_squared_error: 0.1945
Epoch 128/150
23819/23819 [==============================] - 0s 8us/step - los

23819/23819 [==============================] - 0s 9us/step - loss: 0.1985 - mean_squared_error: 0.1985
Epoch 41/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1982 - mean_squared_error: 0.1982
Epoch 42/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1979 - mean_squared_error: 0.1979
Epoch 43/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1984 - mean_squared_error: 0.1984
Epoch 44/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.1977 - mean_squared_error: 0.1977
Epoch 45/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1973 - mean_squared_error: 0.1973
Epoch 46/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1966 - mean_squared_error: 0.1966
Epoch 47/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1965 - mean_squared_error: 0.1965
Epoch 48/150
23819/23819 [==============================] - 0s 9us/step - loss: 0

23819/23819 [==============================] - 0s 10us/step - loss: 0.1907 - mean_squared_error: 0.1907
Epoch 111/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1916 - mean_squared_error: 0.1916
Epoch 112/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.1908 - mean_squared_error: 0.1908
Epoch 113/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.1907 - mean_squared_error: 0.1907
Epoch 114/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.1909 - mean_squared_error: 0.1909
Epoch 115/150
23819/23819 [==============================] - 0s 8us/step - loss: 0.1905 - mean_squared_error: 0.1905
Epoch 116/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1908 - mean_squared_error: 0.1908
Epoch 117/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.1908 - mean_squared_error: 0.1908
Epoch 118/150
23819/23819 [==============================] - 0s 10us/step 

23820/23820 [==============================] - 0s 9us/step - loss: 0.2045 - mean_squared_error: 0.2045
Epoch 30/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.2043 - mean_squared_error: 0.2043
Epoch 31/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.2040 - mean_squared_error: 0.2040
Epoch 32/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.2033 - mean_squared_error: 0.2033
Epoch 33/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.2032 - mean_squared_error: 0.2032
Epoch 34/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.2038 - mean_squared_error: 0.2038
Epoch 35/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.2027 - mean_squared_error: 0.2027
Epoch 36/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.2027 - mean_squared_error: 0.2027
Epoch 37/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.202

23820/23820 [==============================] - 0s 8us/step - loss: 0.1955 - mean_squared_error: 0.1955
Epoch 100/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.1954 - mean_squared_error: 0.1954
Epoch 101/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.1954 - mean_squared_error: 0.1954
Epoch 102/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.1951 - mean_squared_error: 0.1951
Epoch 103/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.1950 - mean_squared_error: 0.1950
Epoch 104/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.1948 - mean_squared_error: 0.1948
Epoch 105/150
23820/23820 [==============================] - 0s 8us/step - loss: 0.1950 - mean_squared_error: 0.1950
Epoch 106/150
23820/23820 [==============================] - 0s 9us/step - loss: 0.1953 - mean_squared_error: 0.1953
Epoch 107/150
23820/23820 [==============================] - 0s 8us/step - los

In [20]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_bigger_model_preds))

0.447140354101107

## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [25]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    
    # init model
    model = models.Sequential()
    
    # input layer (with Dropout)
    model.add(layers.Dropout(0.3, input_shape=(n_features,)))
    
    # first hidden layer
    model.add(layers.Dense(10, input_shape=(n_features,),
                           activation='relu',
                           kernel_regularizer=regularizers.l2(.005)))
    model.add(layers.Dropout(0.3))
    
    # second hidden layer
    model.add(layers.Dense(10, input_shape=(n_features,),
                           activation='relu',
                           kernel_regularizer=regularizers.l2(.005)))
    model.add(layers.Dropout(0.3))
    
    # third hidden layer
    model.add(layers.Dense(10, input_shape=(n_features,),
                           activation='relu',
                           kernel_regularizer=regularizers.l2(.005)))
    model.add(layers.Dropout(0.3))
    
    # output layer
    model.add(layers.Dense(1, activation='linear'))
    
    # compile
    model.compile(optimizer='sgd',
                  loss='mse',
                  metrics=['mse'])
    
    return model

In [26]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model,
                                 epochs=150,
                                 batch_size=256)

In [27]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3,
                                     X_train_all,
                                     y_train_scaled,
                                     cv=5)

Epoch 1/150
23819/23819 [==============================] - 1s 38us/step - loss: 1.1155 - mean_squared_error: 0.9403
Epoch 2/150
23819/23819 [==============================] - 0s 11us/step - loss: 1.0072 - mean_squared_error: 0.8383
Epoch 3/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.8596 - mean_squared_error: 0.6969
Epoch 4/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.8324 - mean_squared_error: 0.6739
Epoch 5/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.8010 - mean_squared_error: 0.6465
Epoch 6/150
23819/23819 [==============================] - 0s 15us/step - loss: 0.7696 - mean_squared_error: 0.6185
Epoch 7/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.7494 - mean_squared_error: 0.6018
Epoch 8/150
23819/23819 [==============================] - 0s 16us/step - loss: 0.7350 - mean_squared_error: 0.5906
Epoch 9/150
23819/23819 [==============================] - 0s 15us/step 

23819/23819 [==============================] - 0s 12us/step - loss: 0.4822 - mean_squared_error: 0.4279
Epoch 72/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4817 - mean_squared_error: 0.4281
Epoch 73/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4786 - mean_squared_error: 0.4255
Epoch 74/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4800 - mean_squared_error: 0.4276
Epoch 75/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4758 - mean_squared_error: 0.4242
Epoch 76/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4697 - mean_squared_error: 0.4186
Epoch 77/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4693 - mean_squared_error: 0.4189
Epoch 78/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4694 - mean_squared_error: 0.4197
Epoch 79/150
23819/23819 [==============================] - 0s 11us/step - lo

23819/23819 [==============================] - 0s 10us/step - loss: 0.4286 - mean_squared_error: 0.3984
Epoch 141/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4399 - mean_squared_error: 0.4099
Epoch 142/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4270 - mean_squared_error: 0.3972
Epoch 143/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4369 - mean_squared_error: 0.4072
Epoch 144/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4268 - mean_squared_error: 0.3973
Epoch 145/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4349 - mean_squared_error: 0.4054
Epoch 146/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4338 - mean_squared_error: 0.4045
Epoch 147/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4260 - mean_squared_error: 0.3968
Epoch 148/150
23819/23819 [==============================] - 0s 10us/s

23819/23819 [==============================] - 0s 11us/step - loss: 0.4944 - mean_squared_error: 0.4322
Epoch 61/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4895 - mean_squared_error: 0.4280
Epoch 62/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4819 - mean_squared_error: 0.4214
Epoch 63/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4882 - mean_squared_error: 0.4286
Epoch 64/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4885 - mean_squared_error: 0.4297
Epoch 65/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4806 - mean_squared_error: 0.4224
Epoch 66/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4876 - mean_squared_error: 0.4300
Epoch 67/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4839 - mean_squared_error: 0.4271
Epoch 68/150
23819/23819 [==============================] - 0s 12us/step - lo

23819/23819 [==============================] - 0s 10us/step - loss: 0.4462 - mean_squared_error: 0.4143
Epoch 130/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4318 - mean_squared_error: 0.4001
Epoch 131/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4275 - mean_squared_error: 0.3958
Epoch 132/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4262 - mean_squared_error: 0.3948
Epoch 133/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4300 - mean_squared_error: 0.3988
Epoch 134/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4310 - mean_squared_error: 0.4000
Epoch 135/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4348 - mean_squared_error: 0.4040
Epoch 136/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4312 - mean_squared_error: 0.4005
Epoch 137/150
23819/23819 [==============================] - 0s 10us/s

...

23819/23819 [==============================] - 0s 17us/step - loss: 0.4315 - mean_squared_error: 0.3950
Epoch 119/150
23819/23819 [==============================] - 0s 19us/step - loss: 0.4260 - mean_squared_error: 0.3897
Epoch 120/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4318 - mean_squared_error: 0.3957
Epoch 121/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4260 - mean_squared_error: 0.3901
Epoch 122/150
23819/23819 [==============================] - 0s 14us/step - loss: 0.4320 - mean_squared_error: 0.3964
Epoch 123/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4245 - mean_squared_error: 0.3889
Epoch 124/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4288 - mean_squared_error: 0.3935
Epoch 125/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4286 - mean_squared_error: 0.3935
Epoch 126/150
23819/23819 [==============================] - 0s 13us/s

23819/23819 [==============================] - 0s 12us/step - loss: 0.5212 - mean_squared_error: 0.4352
Epoch 38/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.5227 - mean_squared_error: 0.4381
Epoch 39/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.5144 - mean_squared_error: 0.4311
Epoch 40/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.5080 - mean_squared_error: 0.4260
Epoch 41/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4996 - mean_squared_error: 0.4188
Epoch 42/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.5167 - mean_squared_error: 0.4373
Epoch 43/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.5093 - mean_squared_error: 0.4311
Epoch 44/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.5046 - mean_squared_error: 0.4277
Epoch 45/150
23819/23819 [==============================] - 0s 10us/step - lo

23819/23819 [==============================] - 0s 10us/step - loss: 0.4406 - mean_squared_error: 0.4039
Epoch 107/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4368 - mean_squared_error: 0.4005
Epoch 108/150
23819/23819 [==============================] - 0s 11us/step - loss: 0.4371 - mean_squared_error: 0.4010
Epoch 109/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4411 - mean_squared_error: 0.4052
Epoch 110/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4427 - mean_squared_error: 0.4070
Epoch 111/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4341 - mean_squared_error: 0.3986
Epoch 112/150
23819/23819 [==============================] - 0s 10us/step - loss: 0.4341 - mean_squared_error: 0.3989
Epoch 113/150
23819/23819 [==============================] - 0s 12us/step - loss: 0.4374 - mean_squared_error: 0.4024
Epoch 114/150
23819/23819 [==============================] - 0s 10us/s

23820/23820 [==============================] - 0s 11us/step - loss: 0.6080 - mean_squared_error: 0.5130
Epoch 26/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.6147 - mean_squared_error: 0.5213
Epoch 27/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.6037 - mean_squared_error: 0.5120
Epoch 28/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.6029 - mean_squared_error: 0.5127
Epoch 29/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.6045 - mean_squared_error: 0.5159
Epoch 30/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.6019 - mean_squared_error: 0.5146
Epoch 31/150
23820/23820 [==============================] - 0s 13us/step - loss: 0.5783 - mean_squared_error: 0.4924
Epoch 32/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.5823 - mean_squared_error: 0.4978
Epoch 33/150
23820/23820 [==============================] - 0s 11us/step - lo

23820/23820 [==============================] - 0s 11us/step - loss: 0.5019 - mean_squared_error: 0.4640
Epoch 96/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.4953 - mean_squared_error: 0.4576
Epoch 97/150
23820/23820 [==============================] - 0s 13us/step - loss: 0.4933 - mean_squared_error: 0.4560
Epoch 98/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.4906 - mean_squared_error: 0.4534
Epoch 99/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.5020 - mean_squared_error: 0.4652
Epoch 100/150
23820/23820 [==============================] - 0s 11us/step - loss: 0.4934 - mean_squared_error: 0.4570
Epoch 101/150
23820/23820 [==============================] - 0s 12us/step - loss: 0.4901 - mean_squared_error: 0.4539
Epoch 102/150
23820/23820 [==============================] - 0s 10us/step - loss: 0.4975 - mean_squared_error: 0.4616
Epoch 103/150
23820/23820 [==============================] - 0s 11us/step 

In [28]:
# RMSE on train data (scaled)
np.sqrt(mean_squared_error(y_train_scaled, cv_dropout_preds))

0.5908933628911801

## Final Evaluation

Now that you have selected a network architecture, tested various regularization procedures and tuned hyperparameters via a validation methodology, it is time to evaluate your final model on the test set. Fit the model using all of the training data using the architecture and hyperparameters that were most effective in your experiments above. Afterwards, measure the overall performance on the hold-out test data which has been left untouched (and hasn't leaked any data into the modeling process)! 

In [29]:
# ⏰ This cell may take several mintes to run
# I'm using the first model that was made

# Initialize model
model = models.Sequential()

# First hidden layer
model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

# Second hidden layer
model.add(layers.Dense(5, activation='relu'))

# Output layer
model.add(layers.Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='SGD', 
              loss='mse',  
              metrics=['mse']) 

# Train the model 
model.fit(X_train_all, 
          y_train_scaled, 
          epochs=150, 
          batch_size=256, 
          verbose=0)

In [30]:
final_preds_scaled = model.predict(X_test_all)

final_preds = ss_y.inverse_transform(final_preds_scaled)

np.sqrt(mean_squared_error(y_test, final_preds))

3852.120015991409

## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 